In [1]:
import re
import time
from colorama import Fore
import openpyxl
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import random
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import math
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
options = Options()

options.add_argument('--disable-application-cache')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--disable-cookies")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument("start-maximized")
options.add_argument('--disable-gpu')
# options.add_argument("--headless")  
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--ignore-urlfetcher-cert-requests')
options.add_argument('--no-first-run')
options.add_argument("--disable-popup-blocking") 


driver = webdriver.Chrome(options=options)

In [4]:
link='https://www.chiviricas.com/'

In [5]:
driver.get(link)

In [33]:
allcards=driver.find_elements(By.XPATH, '//div[@class="pin masonry top_off"]')

In [83]:
page=BeautifulSoup(driver.page_source, 'html.parser')
allcards=page.find_all('div', class_=re.compile(r'\bpin\b.*\bmasonry\b|\bmasonry\b.*\bpin\b'))

In [84]:
titulo=[]
numero=[]
descripcion=[]


In [85]:
for e in allcards:
    try:
        title = e.find('figcaption', class_='titulo').text.strip()
        number = e.find('span', class_='container_img').find('img')['alt'].split(' ')[1].strip()
        description = e.find('span',class_="categorias_basicas").text.strip()
        titulo.append(title)
        numero.append(number)
        descripcion.append(description)
    except Exception as ex:
        print(f'Error processing card: {ex}')

In [64]:
e.find('span', class_='container_img').find('img')['alt']


'Kimberly 8092649685, Prepago en Santo Domingo Este'

In [37]:
for card in allcards:
    try:
        card.click()
        time.sleep(1)
        page=BeautifulSoup(driver.page_source, 'html.parser')
        cardselect=page.find('div', id='ficha_main')
        cardinfo=cardselect.find('section', id='ficha_datos')
        title = cardinfo.find('h1').text.strip()
        if title:
            titulo.append(title)
        else:
            titulo.append(None)
        description = cardselect.find('section', id='ficha_texto').text.strip()
        if description:
            descripcion.append(description)
        else:
            descripcion.append(None)
        try:      
            phone = page.find('div', id='ficha_telefono').get('data-telefono')
        except AttributeError:
            try:
                phone = page.find('div', id='ficha_whatsapp').get('data-telefono')
            except AttributeError:
                phone = page.find('div', id='ficha_telegram').get('data-telefono')
        if phone:
            numero.append(phone)
        else:
            numero.append(None)

        driver.execute_script("document.elementFromPoint(200, 300).click();")
        
    except ElementClickInterceptedException:
        print(  "ElementClickInterceptedException: Retrying click")


ElementClickInterceptedException: Retrying click
ElementClickInterceptedException: Retrying click
ElementClickInterceptedException: Retrying click


UnexpectedAlertPresentException: Alert Text: Error de conexión.
Message: unexpected alert open: {Alert text : Error de conexión.}
  (Session info: chrome=137.0.7151.120)
Stacktrace:
	GetHandleVerifier [0x0x7ff6dccbcda5+78885]
	GetHandleVerifier [0x0x7ff6dccbce00+78976]
	(No symbol) [0x0x7ff6dca79bca]
	(No symbol) [0x0x7ff6dcb21de0]
	(No symbol) [0x0x7ff6dcaf8963]
	(No symbol) [0x0x7ff6dcac16b1]
	(No symbol) [0x0x7ff6dcac2443]
	GetHandleVerifier [0x0x7ff6dcf94eed+3061101]
	GetHandleVerifier [0x0x7ff6dcf8f33d+3037629]
	GetHandleVerifier [0x0x7ff6dcfae592+3165202]
	GetHandleVerifier [0x0x7ff6dccd730e+186766]
	GetHandleVerifier [0x0x7ff6dccdeb3f+217535]
	GetHandleVerifier [0x0x7ff6dccc59b4+114740]
	GetHandleVerifier [0x0x7ff6dccc5b69+115177]
	GetHandleVerifier [0x0x7ff6dccac368+10728]
	BaseThreadInitThunk [0x0x7ff918b3e8d7+23]
	RtlUserThreadStart [0x0x7ff91a0dc34c+44]


In [86]:
sample=({'titulo': titulo,
         'numero': numero,
            'descripcion': descripcion}) 
df = pd.DataFrame(sample)

In [87]:
df.drop_duplicates(inplace=True)

In [90]:
df

,titulo,numero,descripcion
0,"Masajista erótica, saco leche con final de gar...",rubia,"Santo Domingo25 añosRD$ 6.500RD$ 6,5K"
1,"Masajes eróticos, tántricos y profesionales, r...","8294774192,",Santo Domingo28 años
2,Masajista profesional en masajes terapéuticos ...,Fernández,Santo Domingo32 añosRD$ 5.000RD$ 5K
3,"Masajista erótica, con lugar discreto, ducha y...","8099937464,",Santo Domingo22 años
4,Colombiana preciosa dispuesta a complacerte y ...,"8294994415,",Santo Domingo22 años
...,...,...,...
59,Solo masajes eróticos y si quieres fotos pagas...,Mujer,Santo Domingo27 añosRD$ 2.000RD$ 2K
60,Masajista profesional garantizada con licencia.,"Guillén,",Santo Domingo33 añosRD$ 3.000RD$ 3K
61,Masajista y excelente dama de compañía en la p...,Mujer,Punta Cana28 añosRD$ 3.000RD$ 3K
62,"Masajes eróticos, de relajación y más tántrico.",Mujer,"Punta Cana28 añosRD$ 5.600RD$ 5,6K"


In [89]:
df.to_csv('chiviricas_Republica_Dominicana_MAsajistas.csv', index=False)
df.to_excel('chiviricas_Republica_Dominicana_MAsajistas.xlsx', index=False)